# 1- Import the Google Maps API and Foursquare keys and the needed lybraries

In [1]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

import configparser
config = configparser.ConfigParser()
config.read('config.ini')
API_KEY = config['DEFAULT']['API_KEY'] #key for Google API
CLIENT_ID = config['DEFAULT']['CLIENT_ID'] #My Foursquare ID
CLIENT_SECRET = config['DEFAULT']['CLIENT_SECRET'] #My Foursquare Secret
VERSION = config['DEFAULT']['VERSION'] #My Foursquare Version

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import numpy as np
from pandas.io.json import json_normalize

# 2-Calculate distances from the base location to the potential localities using Google Maps API

### Google API parameters

In [3]:
unit='metric'
mode='driving' #can also be walking, bicycling
time_cstr1=3300
time_cstr2=1800
origin='Setubal'
destination1='Azinheira dos Barros, Grândola|Herdade do Azinhal, Grândola|Lousal, Grândola|Monte da Tapada, Grândola|Monte de Mascarenhas, Grândola|Monte de Matos, Grândola|Monte Novo dos Barros, Grândola|Monte Queimado, Grândola|Carvalhal, Grândola|Lagoa do Leite, Grândola|Lagoa Formosa, Grândola|Pego, Grândola|Pinheiro da Cruz, Grândola|Porto Martinho, Grândola|Sequeiros do Paúl, Grândola|Tróia, Grândola|Água Derramada, Grândola|Aldeia do Pico, Grândola|Ameiras de Baixo, Grândola|Bairro Novo de Cadoços, Grândola|Cadocos, Grândola|Canal Caveira, Grândola|Chinita, Grândola|Cilha Pascoal, Grândola|Cruz de Cadoços, Grândola|Fontaínhas, Grândola|Grândola, Grândola|Monte Cadetes, Grândola|Monte dos Cozinheiros, Grândola|Monte Foros Amieiro Incenso, Grândola|Apaulinha, Grândola|Mosqueirões, Grândola|Muda, Grândola|Posto Tirana, Grândola|Quinta do Barahona, Grândola|Quinta do Olival Novo, Grândola|Quinta Velha, Grândola|Recinto da Quinta Velha, Grândola|Regota, Grândola|Várzea Redonda, Grândola|Aderneira, Grândola|Água Boa, Grândola|Água do Porco, Grândola|Água Nova, Grândola|Água Nova de Baixo, Grândola|Alagoas, Grândola|Alagoínha Nova, Grândola|Alqueveirinhos, Grândola|Alto de Santa Clara, Grândola|Alto de Trás-as-Vinhas, Grândola|Alturinha da Sancha, Grândola|Ameira, Grândola|Aranha, Grândola|Aranhadouro, Grândola|Arneiro, Grândola|Arneiro da Vinha, Grândola|Arneiro de Baixo, Grândola|Arrebolada de Cima, Grândola|Bairro Alto, Grândola|Boavista, Grândola|Bomba, Grândola|Botelhas, Grândola|Botinha Nova, Grândola|Brejo da Lagoa, Grândola|Cabeça Gorda, Grândola|Cai Logo, Grândola|Canas, Grândola|Cantos, Grândola|Cardal do Barreiro, Grândola|Cardela, Grândola|Cardela do Meio, Grândola|Carneiros de Baixo, Grândola|Carquejeira do Cabeço, Grândola|Casa Branca, Grândola|Casa Nova, Grândola|Casa Nova da Amoreira, Grândola|Casa Nova da Marrã, Grândola|Casa Nova da Sancha, Grândola|Casa Nova das Várzeas, Grândola|Casa Nova do Juncal, Grândola|Casa Nova do Seiceiro, Grândola|Casal Ventoso, Grândola|Casão da Quinta, Grândola|Casas Altas, Grândola|Casas Brancas, Grândola|Casas Velhas, Grândola|Casinha da Corte, Grândola|Casinha Nova da Eira, Grândola|Casqueiro, Grândola|Caveira, Grândola|Caveira do Sul, Grândola|Cerca da Fonte, Grândola|Cerca de Baixo, Grândola|Cerca do Poço, Grândola|Chaminé, Grândola|Chão dos Barreiros, Grândola|Chaparral, Grândola|Charnequinha do Poço, Grândola'
destination2='Cilha Centeno, Grândola|Cilha de Baixo, Grândola|Cilha do Areeiro, Grândola|Cilha Nova, Grândola|Coitada de Cima, Grândola|Condutos, Grândola|Corte de Baixo, Grândola|Corte de São Simão, Grândola|Cortes, Grândola|Courela da Cerca, Grândola|Courela da Sobreirinha, Grândola|Courela de Cima, Grândola|Courela de Santa Marinha, Grândola|Courela do Brejinho da Lagoa, Grândola|Courela do Pereiro, Grândola|Courela do Pinhal, Grândola|Courela dos Cabeços, Grândola|Courela dos Pereiros, Grândola|Courelas Novas, Grândola|Cumeadas, Grândola|Ervideira, Grândola|Ervilhal, Grândola|Escaravilheira, Grândola|Escavadada, Grândola|Esteveira, Grândola|Fernão Afonso, Grândola|Figueiral, Grândola|Figueirinha da Sancha, Grândola|Fontaínhas da Altura, Grândola|Fontaínhas da Barroca, Grândola|Fontaínhas da Ponte, Grândola|Fontaínhas da Várzea, Grândola|Fontaínhas de Baixo, Grândola|Fontaínhas de Cima, Grândola|Fontaínhas do Mar, Grândola|Fontaínhas do Meio, Grândola|Fonte Coelho, Grândola|Fonte do Cortiço, Grândola|Forno da Telha, Grândola|Galega, Grândola|Graciosa, Grândola|Herdade do Pinheirinho, Grândola|Lagoa do Brejinho, Grândola|Lagoa da Coitada, Grândola|Lagoa de Melides, Grândola|Lagoazinha Nova, Grândola|Leal da Várzea, Grândola|Leal das Cercas, Grândola|Lezíria, Grândola|Marmeleiros, Grândola|Marquinhos, Grândola|Marrã do Meio, Grândola|Martins, Grândola|Medronal, Grândola|Meia Légua, Grândola|Melides, Grândola|Moinho de Cima, Grândola|Moinho do Meio, Grândola|Moinho do Vau, Grândola|Moinho Velho, Grândola|Monte Alto, Grândola|Monte da Capitã, Grândola|Monte da Chã, Grândola|Monte da Guadiana, Grândola|Monte da Lezíria, Grândola|Monte da Oliveira, Grândola|Monte da Pereira, Grândola|Monte da Saibreira, Grândola|Monte da Venda, Grândola|Monte das Oliveirinhas, Grândola|Monte do Casão, Grândola|Monte do Nogal, Grândola|Monte do Nosgueiro, Grândola|Monte do Outeiro, Grândola|Monte do Pinheiro, Grândola|Monte do Rosmaninho, Grândola|Monte do Tabarneco, Grândola|Monte dos Carneiros, Grândola|Monte Novo da Chã, Grândola|Monte Novo da Sancha, Grândola|Monte Trás das Vinhas, Grândola|Montum, Grândola|Mortais Novos, Grândola|Nosgueiro, Grândola|Outeirinho da Sancha, Grândola|Outeiro das Sobreiras, Grândola|Outeiro do Castelinho, Grândola|Outeiro do Chaparrinho, Grândola|Palhotas, Grândola|Palhotas de Baixo, Grândola|Palhotas de Cima, Grândola|Pardieiro Novo, Grândola'
destination3='Parque de Campismo da Galé, Grândola|Passagem da Sancha, Grândola|Peçarreira, Grândola|Peguinhos da Cilha, Grândola|Pereira Verde, Grândola|Pereirinho, Grândola|Pesatais, Grândola|Pesatais de Baixo, Grândola|Pinhal de Baixo, Grândola|Pinhal de Cima, Grândola|Pinhal Novo, Grândola|Pinhal Verde, Grândola|Pinhalinho, Grândola|Pinheirinhos, Grândola|Pinheiro da Chave, Grândola|Pinheiro da Rosa, Grândola|Pinheiro da Várzea, Grândola|Pinheiro do Cravo, Grândola|Poças de Água, Grândola|Poças de Água do Meio, Grândola|Pomar Novo, Grândola|Pomarinho, Grândola|Praia de Melides, Grândola|Queimada, Grândola|Queimada de Baixo, Grândola|Quinta da Cardela, Grândola|Quinta da Laranjeira, Grândola|Quinta do Poço, Grândola|Quinta do Vale Figueira, Grândola|Quinta Nova das Fontaínhas, Grândola|Quintinha do Arco Iris, Grândola|Regouga, Grândola|Regouga de Cima, Grândola|Remedeio, Grândola|Rosmaninho, Grândola|Saibreira, Grândola|Salvada de Baixo, Grândola|Samoucal, Grândola|Sancha, Grândola|Sancha Nova, Grândola|Sanchinha do Meio, Grândola|Santo António, Grândola|Seraminheiro, Grândola|Serrado do Nogal, Grândola|Sesmarias de Baixo, Grândola|Sesmarias de Cima, Grândola|Sesmarias do Meio, Grândola|Sesmarias do Tanque, Grândola|Silveiras, Grândola|Sobreiras Altas, Grândola|Sobreirinha da Regouga, Grândola|Sol Posto, Grândola|Tarrajana, Grândola|Trás das Vinhas, Grândola|Trás dos Quintais, Grândola|Vale da Horta, Grândola|Vale da Lagoa, Grândola|Vale da Pereira Verde, Grândola|Vale das Casas Altas, Grândola|Vale das Varas, Grândola|Vale de Figueira, Grândola|Vale de Mouro, Grândola|Vale do Poço, Grândola|Vale dos Cardos, Grândola|Vale Paulino, Grândola|Vale Seiceiro, Grândola|Vale Travesso, Grândola|Vale Travesso Novo, Grândola|Valência de Baixo, Grândola|Valência do Meio, Grândola|Valência Nova do Sul, Grândola|Valinho da Estrada, Grândola|Várzea do Outeiro da Mina, Grândola|Várzeas, Grândola|Várzeas de Baixo, Grândola|Várzeas de Cima, Grândola|Vigia, Grândola|Vigia do Alto, Grândola|Santa Margarida da Serra, Grândola|Outeiro do André, Grândola|Vale da Cobra, Grândola|Ferrarias, Grândola|Breijinho de Água, Grândola|Miranda, Grândola|Pintos, Grândola|Porto Carvalho, Grândola|Brejo da Amada, Grândola|Marrã de Baixo, Grândola|Monte Vale das Lamas, Grândola|Cilha do Centeio, Grândola|Courela da Fonte, Grândola|Ameiras de Cima, Grândola|Boiças, Grândola|Ameiras do Incenso, Grândola|Murteiras, Grândola|Cerca da Zorra, Grândola|Barradas, Grândola|Galé, Grândola|Comporta, Grândola'
destination=destination1
url1='https://maps.googleapis.com/maps/api/distancematrix/json?units='+unit+'&origins='+origin+'&destinations='+destination+'&mode='+mode+'&key='+API_KEY
destination=destination2
url2='https://maps.googleapis.com/maps/api/distancematrix/json?units='+unit+'&origins='+origin+'&destinations='+destination+'&mode='+mode+'&key='+API_KEY#print(url)
destination=destination3
url3='https://maps.googleapis.com/maps/api/distancematrix/json?units='+unit+'&origins='+origin+'&destinations='+destination+'&mode='+mode+'&key='+API_KEY

### Get the data from Google API (url's have a limited size, so I had to split it in 3 requests)

In [4]:
website_url1 = requests.get(url1).text
website_url2 = requests.get(url2).text
website_url3 = requests.get(url3).text
json_data1 = json.loads(website_url1)
json_data2 = json.loads(website_url2)
json_data3 = json.loads(website_url3)

### Join the 3 results dataframes (using append)

In [5]:
data1=json_data1['rows'][0]['elements']
data2=json_data2['rows'][0]['elements']
data3=json_data3['rows'][0]['elements']
df = pd.DataFrame.from_dict(json_normalize(data1), orient='columns')
df2 = pd.DataFrame.from_dict(json_normalize(data2), orient='columns')
df3 = pd.DataFrame.from_dict(json_normalize(data3), orient='columns')
df=df.append(df2, ignore_index=True)
df=df.append(df3, ignore_index=True)
df=df.drop('status', axis=1)
destinos1=json_data1['destination_addresses']
destinos2=json_data2['destination_addresses']
destinos3=json_data3['destination_addresses']
destinos = pd.DataFrame.from_dict(destinos1, orient='columns')
destinos=destinos.append(destinos2, ignore_index=True)
destinos=destinos.append(destinos3, ignore_index=True)
df['destinos']=destinos
df=df.drop_duplicates(keep=False)

### Filter the data according to the initial constraint (duration) and clean duplicates and other abnormal results

In [6]:
desf = df[df['duration.value'] < time_cstr1]
locations2drop=['Lisbon','Lisboa','Barreiro']
#,'Várzea de Grândola','R. Anchieta','R. Vasco da Gama','Rua Padre Barnabé Afonso Barradas','R. Infante Dom Henrique','R. 1 de Maio','Bairro do Arneiro','Rua da Saibreira','Grandola, 7570-113','IC1, R. Dom Nuno Álvares Pereira']
for locat in locations2drop:
    desf=desf[~desf['destinos'].str.contains(locat)]
desf=desf.reset_index(drop=True)
desf

,distance.text,distance.value,duration.text,duration.value,destinos
0,92.3 km,92303.0,54 mins,3244.0,"Azinheira dos Barros e São Mamede do Sádão, Po..."
1,77.9 km,77917.0,53 mins,3169.0,"Aldeia do Pico, 7570 Grândola, Portugal"
2,75.6 km,75592.0,50 mins,2995.0,"Ameiras de Baixo, 7570 Grândola, Portugal"
3,87.1 km,87085.0,53 mins,3164.0,"Apaúlinha, 7570 Grândola, Portugal"
4,83.6 km,83644.0,53 mins,3160.0,"Estrada Aldeia Do Futuro, Lote B R/C, Núcleo E..."
5,87.5 km,87520.0,54 mins,3267.0,"Bairro do Arneiro, 7570-299 Grândola, Portugal"
6,82.5 km,82492.0,50 mins,3014.0,"IC1, R. Dom Nuno Álvares Pereira Lote 2, 7570-..."
7,81.1 km,81133.0,49 mins,2962.0,"R. 1 de Maio 1, 7570 Grândola, Portugal"
8,67.8 km,67828.0,45 mins,2690.0,"Castelo Ventoso, 7580 Alcácer do Sal, Portugal"
9,83.6 km,83603.0,54 mins,3213.0,"7570 Grândola, Portugal"


### Add geographical coordinates to each location (using Google Geocoding API)

In [7]:
tam=desf.shape[0]
lati=[None]*tam
longi=[None]*tam

for i in range(0,tam):
    address=desf['destinos'][i]
    urlocat='https://maps.googleapis.com/maps/api/geocode/json?address='+address+'&key='+API_KEY
    website_urlocat = requests.get(urlocat).text
    json_data_locat = json.loads(website_urlocat)
    lati[i]=json_data_locat['results'][0]['geometry']['location']['lat']
    longi[i]=json_data_locat['results'][0]['geometry']['location']['lng']
desf['lat']=lati
desf['long']=longi
print(desf.shape)
desf.head()

(23, 7)


,distance.text,distance.value,duration.text,duration.value,destinos,lat,long
0,92.3 km,92303.0,54 mins,3244.0,"Azinheira dos Barros e São Mamede do Sádão, Po...",38.120061,-8.418422
1,77.9 km,77917.0,53 mins,3169.0,"Aldeia do Pico, 7570 Grândola, Portugal",38.232677,-8.574966
2,75.6 km,75592.0,50 mins,2995.0,"Ameiras de Baixo, 7570 Grândola, Portugal",38.220974,-8.556894
3,87.1 km,87085.0,53 mins,3164.0,"Apaúlinha, 7570 Grândola, Portugal",38.182715,-8.599024
4,83.6 km,83644.0,53 mins,3160.0,"Estrada Aldeia Do Futuro, Lote B R/C, Núcleo E...",38.176256,-8.566297


### Map of the potencial locations of the house

In [8]:
neighborhood_latitude=38.175375
neighborhood_longitude=-8.565225
house_lati=38.5173637
house_longi=-8.8963407
map_locations = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=10,width='50%',height='50%')

# add markers to map
for lat, lng, name in zip(desf['lat'], desf['long'], desf['destinos']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_locations) 
    
label = 'Base Location: Setúbal'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
        [house_lati, house_longi],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_locations)  
    
map_locations

# 3-Get the Foursquare venues (beaches)

### Foursquare parameters

In [9]:
### Location around which we'll look for the venues
neighborhood_latitude=38.175375
neighborhood_longitude=-8.565225
LIMIT=100
radius=35000
category='4bf58dd8d48988d1e2941735' #venue category is "beach"
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    VERSION,  
    radius,
    LIMIT,
    category)

### Get the Foursquare results

In [10]:
results = requests.get(url).json()

### Function to extract the category of the venue

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Parse and filter the results

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

venue_cat="Beach"
nearby_venues=nearby_venues[nearby_venues['categories'].str.match(venue_cat)]

venue_drop=['Carvalhal','Lagoa de Melides','Lagoa de Santo André','Praia de São Torpes']
for n in venue_drop:
    nearby_venues=nearby_venues[~nearby_venues['name'].str.match(n)]

nearby_venues=nearby_venues.reset_index(drop=True)

nearby_venues

,name,categories,lat,lng
0,Praia do Pego,Beach,38.292676,-8.777960
1,Praia da Lagoa de Santo André,Beach,38.113622,-8.796523
2,Praia da Comporta,Beach,38.380556,-8.803267
3,Praia do Carvalhal,Beach,38.305868,-8.779559
4,Praia da Galé,Beach,38.204665,-8.775190
5,Praia de Melides,Beach,38.129052,-8.790572
6,Praia da Aberta Nova,Beach,38.177456,-8.780651


### Map of the beach locations

In [13]:
map_beaches = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=10,width='50%',height='50%')

# add markers to map
for lat, lng, name in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_beaches)  

for lat, lng, name in zip(desf['lat'], desf['long'], desf['destinos']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_beaches) 
label = 'Base Location: Setúbal'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
        [house_lati, house_longi],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_beaches)  
map_beaches
#map_locations

# 4- Find the locations which are closest to the beach

### Build the origin string

In [14]:
num_origin=nearby_venues.shape[0]
originb=''
for n in range(0,num_origin):
    if n==0:
        originb=nearby_venues['name'][n]
    else:
        originb=originb+',grandola|'+nearby_venues['name'][n]

### Build the destination string

In [15]:
num_destination=desf.shape[0]
split_destination=round(num_destination/2) #must split the destinations in 2 strings because of the Google API results limit
destinationb1=''
destinationb2=''
b1=0
b2=0
for n in range(0,num_destination):
    if n==0:
        destinationb1=desf['destinos'][n]
        b1=b1+1
    elif n<split_destination:
        destinationb1=destinationb1+'|'+desf['destinos'][n]
        b1=b1+1
    else:
        destinationb2=destinationb2+'|'+desf['destinos'][n]
        b2=b2+1
print('b1: ',b1,' b2:',b2)

b1:  12  b2: 11


### Get the travel length duration from Google Maps API

In [16]:
origin=originb
destinationb1
urlb='https://maps.googleapis.com/maps/api/distancematrix/json?units='+unit+'&origins='+origin+'&destinations='+destinationb1+'&mode='+mode+'&key='+API_KEY
website_urlb1 = requests.get(urlb).text
json_datab1 = json.loads(website_urlb1)
destination=destinationb2
urlb='https://maps.googleapis.com/maps/api/distancematrix/json?units='+unit+'&origins='+origin+'&destinations='+destinationb2+'&mode='+mode+'&key='+API_KEY
website_urlb2 = requests.get(urlb).text
json_datab2 = json.loads(website_urlb2)
duration = pd.DataFrame.from_dict(json_normalize(data2), orient='columns')

### Create a dataframe with the results from the Google Distance Matrix

In [17]:
df_or=nearby_venues['name'] #origins
df_des=desf['destinos'] #destinations
df_or=nearby_venues['name'] #origins
df_des=desf['destinos'] #destinations
dm=pd.DataFrame(columns=['Origin', 'Destination', 'Time'], index=range(1,num_origin*num_destination+1))
row_dm=1
#first distance matrix
for i in range(0,num_origin):
    for j in range(0,split_destination-1):
        dm['Origin'][row_dm]=df_or[i]
        dm['Destination'][row_dm]=df_des[j]
        dm['Time'][row_dm]=json_datab1['rows'][i]['elements'][j]['duration']['value']/60
        row_dm=row_dm+1
#second distance matrix
for i in range(0,num_origin):
    for j in range(split_destination,num_destination):
        dm['Origin'][row_dm]=df_or[i]
        dm['Destination'][row_dm]=df_des[j]
        dm['Time'][row_dm]=json_datab2['rows'][i]['elements'][j-split_destination]['duration']['value']/60
        row_dm=row_dm+1
print(dm.shape)

(161, 3)


### Filter the locations which are to far from the beach

In [18]:
dm=dm[dm['Time']<30]
#dm=dm[ignore_index=True]
dm=dm.reset_index(drop=True)
print(dm.shape)
dm.head()

(101, 3)


,Origin,Destination,Time
0,Praia do Pego,"Aldeia do Pico, 7570 Grândola, Portugal",27.6833
1,Praia do Pego,"Ameiras de Baixo, 7570 Grândola, Portugal",27.0333
2,Praia do Pego,"Apaúlinha, 7570 Grândola, Portugal",22.5667
3,Praia do Pego,"Estrada Aldeia Do Futuro, Lote B R/C, Núcleo E...",26.4667
4,Praia do Pego,"Bairro do Arneiro, 7570-299 Grândola, Portugal",27.9


### Create DataFrames with final locations

In [19]:
unique_beaches=pd.DataFrame(columns=['Beaches'],data=dm['Origin'].unique())
unique_locations=pd.DataFrame(columns=['Locations'],data=dm['Destination'].unique())
final_beaches=nearby_venues.loc[nearby_venues['name'].isin(unique_beaches['Beaches'])]
final_locations=desf.loc[desf['destinos'].isin(unique_locations['Locations'])]
final_locations=final_locations.drop(['distance.text','duration.text'], axis=1)
final_locations['duration.value']=(final_locations['duration.value']/60).round()
final_locations.rename(index=str, columns={"distance.value": "distance_home", "duration.value": "time_home"}).head()

,distance_home,time_home,destinos,lat,long
1,77917.0,53.0,"Aldeia do Pico, 7570 Grândola, Portugal",38.232677,-8.574966
2,75592.0,50.0,"Ameiras de Baixo, 7570 Grândola, Portugal",38.220974,-8.556894
3,87085.0,53.0,"Apaúlinha, 7570 Grândola, Portugal",38.182715,-8.599024
4,83644.0,53.0,"Estrada Aldeia Do Futuro, Lote B R/C, Núcleo E...",38.176256,-8.566297
5,87520.0,54.0,"Bairro do Arneiro, 7570-299 Grândola, Portugal",38.172179,-8.558255


### Complete the dataframe with the closest beach and time of travel to the beach

In [20]:
i=0
time_beach=[None]*unique_locations.shape[0]
closest_beach=[None]*unique_locations.shape[0]
for destino in final_locations['destinos']:
    gb=dm[dm['Destination']==destino]
    gb=gb.sort_values(by =['Time'])
    gb.iloc[0,:]
    closest_beach[i]=gb.iloc[0,:][0]
    time_beach[i]=round(gb.iloc[0,:][2],2)
    i=i+1
final_locations['time_beach']=time_beach
final_locations['closest_beach']=closest_beach
final_locations=final_locations.reset_index(drop=True)
print(final_locations.shape)
final_locations.head()

(19, 7)


,distance.value,duration.value,destinos,lat,long,time_beach,closest_beach
0,77917.0,53.0,"Aldeia do Pico, 7570 Grândola, Portugal",38.232677,-8.574966,25.37,Praia do Carvalhal
1,75592.0,50.0,"Ameiras de Baixo, 7570 Grândola, Portugal",38.220974,-8.556894,24.70,Praia do Carvalhal
2,87085.0,53.0,"Apaúlinha, 7570 Grândola, Portugal",38.182715,-8.599024,20.23,Praia do Carvalhal
3,83644.0,53.0,"Estrada Aldeia Do Futuro, Lote B R/C, Núcleo E...",38.176256,-8.566297,24.15,Praia do Carvalhal
4,87520.0,54.0,"Bairro do Arneiro, 7570-299 Grândola, Portugal",38.172179,-8.558255,25.58,Praia do Carvalhal


### Plot the locations and the beaches in the map

In [21]:
map_final = folium.Map(location=[neighborhood_latitude, neighborhood_longitude],zoom_start=10,width='50%',height='50%' )

# add markers to map
for lat, lng, name in zip(final_beaches['lat'], final_beaches['lng'], final_beaches['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_final)  

for lat, lng, name, beach, time in zip(final_locations['lat'], final_locations['long'], final_locations['destinos'], final_locations['closest_beach'],final_locations['time_beach']):
    label = 'location: {}, {} min, beach: {}'.format(name, time, beach)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_final)
label = 'Base Location: Setúbal'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
        [house_lati, house_longi],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_final)  
map_final

# 5- Cluster the locations with k-means

### Standardizing, fitting the model and predicting the clusters

In [22]:
kclusters = 3
from sklearn.preprocessing import StandardScaler
clustering = final_locations[['distance.value','duration.value','time_beach']]
scaler = StandardScaler().fit(clustering)

# Standardize the columns.
standardized_data = final_locations.copy()
#print(standardized_data)
standardized_columns = scaler.transform(clustering)
standardized_data[['distance.value','duration.value','time_beach']] = standardized_columns
#print(standardized_data)
# Fit the model.
from sklearn.cluster import KMeans
model = KMeans(n_clusters=kclusters).fit(standardized_columns)

# Get results.
final_locations['cluster'] = model.predict(standardized_columns)
clustering=final_locations

### Plot the clusters in the map

In [23]:
# create map
map_clusters = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=12,width='50%',height='50%')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, loc, cluster in zip(clustering['lat'], clustering['long'], clustering['destinos'],clustering['cluster']):
    label = folium.Popup(str(loc)+' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

### After severall tries of cluster numbers, I keep 3 clusters:
##### -Cluster 0 (red): Locations farthest and longest from home, but faster to the beach
##### -Cluster 1 (blue): Locations closest and fastest from home, but longer to the beach
##### -Cluster 2 (green): Locations farthest and longest from home and longer from the beach

### The average values of the clusters (keep in mind that I only used the distance, duration and time to the beach as features in k means)

In [24]:
gb=clustering.groupby(['cluster']).mean()
gb

,distance.value,duration.value,lat,long,time_beach
cluster,,,,,
0,84556.00,53.142857,38.176825,-8.572745,23.464286
1,79445.25,51.250000,38.213884,-8.564492,25.137500
2,84798.00,53.875000,38.181069,-8.566199,25.645000


### The complete cluster distribution

In [25]:
clustering.sort_values(by=['cluster'])

,distance.value,duration.value,destinos,lat,long,time_beach,closest_beach,cluster
18,83815.0,53.0,"Rua Padre Barnabé Afonso Barradas, 7570 Grândo...",38.178161,-8.572346,23.75,Praia do Carvalhal,0
15,83738.0,53.0,"Av. Jorge de Vasconcelos Nunes 98, 7570-239 Gr...",38.177534,-8.565034,24.40,Praia do Carvalhal,0
2,87085.0,53.0,"Apaúlinha, 7570 Grândola, Portugal",38.182715,-8.599024,20.23,Praia do Carvalhal,0
3,83644.0,53.0,"Estrada Aldeia Do Futuro, Lote B R/C, Núcleo E...",38.176256,-8.566297,24.15,Praia do Carvalhal,0
14,85231.0,53.0,"Várzea de Grândola (IC1, saída para a N120 - S...",38.171324,-8.570365,23.65,Praia do Carvalhal,0
8,84776.0,53.0,"R. Anchieta, 7570 Grândola, Portugal",38.175531,-8.569848,23.92,Praia do Carvalhal,0
6,83603.0,54.0,"7570 Grândola, Portugal",38.176256,-8.566297,24.15,Praia do Carvalhal,0
0,77917.0,53.0,"Aldeia do Pico, 7570 Grândola, Portugal",38.232677,-8.574966,25.37,Praia do Carvalhal,1
5,82492.0,50.0,"IC1, R. Dom Nuno Álvares Pereira Lote 2, 7570-...",38.185956,-8.568326,24.20,Praia do Carvalhal,1
1,75592.0,50.0,"Ameiras de Baixo, 7570 Grândola, Portugal",38.220974,-8.556894,24.70,Praia do Carvalhal,1
